# このノートブックについて

### 概要
1. 2020 年現在において高い性能を持つアルゴリズム (SAC) を用いて Pendulum の強化学習を行う
1. 2017 年に発表されたアルゴリズムである PPO でも同様に Pendulum の強化学習を行う
1. SAC と PPO の結果を比較する

### 身につけられる知識
1. アルゴリズムを切り替える方法
1. 実験の管理
1. 性能指標の確認

In [ ]:
from distutils import dir_util
import os.path

# Azure Machine Learning core imports
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Dataset
from azureml.core.compute import ComputeInstance
from azureml.core.compute_target import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.widgets import RunDetails

import consts

%matplotlib inline

# Pendulum について

Pendulum は振り子に適切な力を加え、振り子を立たせることが目的となる問題です。

<table style="width:25%">
  <tr>
    <th>
      <img src="./images/pendulum.png" alt="Pendulum image" /> 
    </th>
  </tr>
  <tr>
      <th><p>Pendulum</p></th>
  </tr>
</table>

<img src="./images/about_pendulum_1.png" alt="about pendulum" />

# Workspace と Compute target の設定
1つめのチュートリアルで説明済みなので、詳細は省略します

### Workspace の取得

In [ ]:
# Workspace の取得
interactive_auth = InteractiveLoginAuthentication(tenant_id=consts.tenant_id)

ws = Workspace(subscription_id=consts.subscription_id,
               resource_group=consts.resource_group,
               workspace_name=consts.workspace_name,
               auth=interactive_auth)
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Compute target として Computing cluster を指定する

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
import os

# Cluster の名前とサイズを選択する
compute_name = consts.cc_cpu_name
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_D2_V2"

if compute_name in ws.compute_targets:
    print("次の Cluster が見つかりました: " + compute_name)
    compute_target = ws.compute_targets[compute_name]
else:
    print("新しい Cluster を作成します")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes, 
        max_nodes=compute_max_nodes)
        
    # Cluster の作成
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

# 訓練
アルゴリズムとして SAC を使い Pendulum を解く Agent の訓練を行います。<br>
比較のため、PPO を使った訓練も行います。

### SAC による訓練

#### 注意点
2020 年 12 月現在、連続値問題に対して SAC を適用しようとすると必用なライブラリが不足しているため、エラーが発生します。<br>
このエラーを回避するためには、以下を行う必用があります。
* ray のバージョンを 0.8.7 以上にする
* TensorFlow のバージョンを上げる
* TensorFlow Probability をインストールする

In [ ]:
training_algorithm = "SAC"
rl_environment = "Pendulum-v0"  # 新しい課題を使用する

script_params = {

    # 訓練アルゴリズム。ここでは SAC
    "--run": training_algorithm,  # "SAC"
    
    # 環境。ここでは Pendulum-v0
    "--env": rl_environment,  # "Pendulum-v0"
    
    # 訓練に関係する config を設定する
    "--config": '\'{"num_gpus": 0, "num_workers": 1}\'',  # gpu は使わず、simulation は並列実行せず1つのプロセスで行う
    
    # 訓練の終了条件
    # Simulationをスタート (リセット) してから終了するまでをエピソードという
    # ここでは複数回エピソードの平均報酬が -500 に達するか、訓練時間が 900秒を超えると訓練を終了する
    "--stop": '\'{"episode_reward_mean": -500, "time_total_s": 900}\'', 
    
    # チェックポイント (モデルの重みなど) の作成頻度
    # ここでは 2エピソード毎に作成する
    "--checkpoint-freq": 50,
    
    # 訓練終了時にもチェックポイントを作成する。値は空欄でOK。
    "--checkpoint-at-end": "",
    
    # Tensorboard で開くことのできるログの場所を指定する
    "--local-dir": './logs'
}

training_estimator = ReinforcementLearningEstimator(

    # 訓練スクリプトが入っているフォルダを指定
    source_directory='files',
    
    # 訓練スクリプトのファイル名
    entry_script='cartpole_training.py',
    
    # 上で定義した訓練スクリプトへ渡す引数
    # どのようにパースされるかは cartpole_training.py を参照
    script_params=script_params,
    
    # compute target を指定する。ここではこのノートブックを開いている compute instance を指定する
    compute_target=compute_target,
    
    # 今現在は Ray() で固定
    rl_framework=Ray(),
    
    # pip を使ってライブラリを追加する。最新の ray を使用する。
    pip_packages=["ray[rllib]==0.8.7", "tensorflow==2.1.0", "tensorflow_probability==0.9.0"]
)

In [ ]:
# 実験の作成
experiment_name = 'Demo02-Pendulum-SAC-Train'  # 任意の名称を入力
exp = Experiment(workspace=ws, name=experiment_name)

# 訓練を実行する
training_run = exp.submit(training_estimator)

# 訓練をモニタリングする
RunDetails(training_run).show()

# 訓練完了を待つ
training_run.wait_for_completion()

### PPO による訓練

In [ ]:
script_params = {
    
    # 訓練アルゴリズム。ここでは SAC
    "--run": "PPO",  # 比較のため、PPO を使用
    
    # 環境。ここでは Pendulum-v0
    "--env": rl_environment,  # "Pendulum-v0"
    
    # 訓練に関係する config を設定する
    "--config": '\'{"num_gpus": 0, "num_workers": 1}\'',  # gpu は使わず、simulation は並列実行せず1つのプロセスで行う
    
    # 訓練の終了条件
    # Simulationをスタート (リセット) してから終了するまでをエピソードという
    # ここでは複数回エピソードの平均報酬が -500 に達するか、訓練時間が 900秒を超えると訓練を終了する
    "--stop": '\'{"episode_reward_mean": -500, "time_total_s": 900}\'', 
    
    # チェックポイント (モデルの重みなど) の作成頻度
    # ここでは 2エピソード毎に作成する
    "--checkpoint-freq": 50,
    
    # 訓練終了時にもチェックポイントを作成する。値は空欄でOK。
    "--checkpoint-at-end": "",
    
    # Tensorboard で開くことのできるログの場所を指定する
    "--local-dir": './logs'
}

training_estimator_ppo = ReinforcementLearningEstimator(

    # 訓練スクリプトが入っているフォルダを指定
    source_directory='files',
    
    # 訓練スクリプトのファイル名
    entry_script='cartpole_training.py',
    
    # 上で定義した訓練スクリプトへ渡す引数
    # どのようにパースされるかは cartpole_training.py を参照
    script_params=script_params,
    
    # compute target を指定する。ここではこのノートブックを開いている compute instance を指定する
    compute_target=compute_target,
    
    # 今現在は Ray() で固定
    rl_framework=Ray(),
    
    # pip を使ってライブラリを追加する。最新の ray を使用する。
    pip_packages=["ray[rllib]==0.8.7", "tensorflow==2.1.0", "tensorflow_probability==0.9.0"]
)

In [ ]:
# 実験の作成
experiment_name_ppo = 'Demo02-Pendulum-PPO-Train'  # 任意の名称を入力
exp_ppo = Experiment(workspace=ws, name=experiment_name_ppo)

# 訓練を実行する
training_run_ppo = exp_ppo.submit(training_estimator_ppo)

# 訓練をモニタリングする
RunDetails(training_run_ppo).show()

# 訓練完了を待つ
training_run_ppo.wait_for_completion()

### 結果の比較
上記の実験は平均報酬が -500 に達するか、訓練時間が300秒を超えると終了します。<br>
SAC は目標とする平均報酬を達成出来た一方、PPO は訓練時間が 900 秒以上になり、終了したと思います。<br>
以下は、それぞれの平均報酬の推移の例です。

<img src="./images/pendulum_sac_vs_ppo.png" alt="SAC vs. PPO" /> 